This script performs regression forecasting on financial or economic time series data using Ridge and Lasso models.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import shapiro
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor

sns.set_theme(style="whitegrid")

In [ ]:
#Load and Prepare Data
data_path = "data/monthly_data.csv"     # your dataset path
target_column = "Target_Return"         # change to your target variable

df = pd.read_csv(data_path)

if target_column not in df.columns:
    raise ValueError(f"Target column '{target_column}' not found in data.")

X = df.drop(columns=[target_column])
y = df[target_column]

train_size = int(len(df) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]


In [ ]:
#VIF Analysis (Multicollinearity)
vif_data = pd.DataFrame({
    "Feature": X.columns,
    "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})
print("\nVIF Analysis:")
print(vif_data)

high_vif = vif_data[vif_data["VIF"] > 5]
if not high_vif.empty:
    print("\n High Multicollinearity Found in:")
    print(high_vif)


# %% [4] Define Helper Function
def evaluate_model(model, name, X_train, X_test, y_train, y_test):
    """Fit, evaluate, and visualize Ridge/Lasso models."""
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    print(f"\n=== {name} Evaluation ===")
    print(f"Train R²: {train_r2:.4f}")
    print(f"Test R²: {test_r2:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")

    # Coefficient Plot
    coefficients = pd.DataFrame(model.coef_, index=X_train.columns, columns=["Coefficient"])
    plt.figure(figsize=(14, 8))
    sns.barplot(x=coefficients["Coefficient"], y=coefficients.index, palette="coolwarm")
    plt.title(f"{name} Regression Coefficients")
    plt.grid(True)
    plt.show()

    # Actual vs Predicted - Line Plot
    plt.figure(figsize=(14, 7))
    plt.plot(y_test.values, label="Actual", marker='o')
    plt.plot(y_test_pred, label="Predicted", linestyle='--', marker='x')
    plt.legend()
    plt.title(f"{name} - Actual vs Predicted (Monthly Return)")
    plt.grid(True)
    plt.show()

    # Actual vs Predicted - Bar Plot
    plt.figure(figsize=(14, 7))
    plt.bar(range(min(15, len(y_test))), y_test.iloc[:15], label="Actual", color="skyblue")
    plt.bar(range(min(15, len(y_test))), y_test_pred[:15], label="Predicted", 
            color="orange", alpha=0.7)
    plt.legend()
    plt.title(f"{name} - Actual vs Predicted (First 15 Observations)")
    plt.grid(True)
    plt.show()

    # Residuals
    residuals = y_test.values - y_test_pred
    plt.figure(figsize=(12, 7))
    plt.scatter(y_test_pred, residuals, edgecolor='k')
    plt.axhline(0, color='red', linestyle='--')
    plt.title(f"{name} Residual Plot")
    plt.grid(True)
    plt.show()

    # Diagnostic Tests
    shapiro_test = shapiro(residuals)
    bp_test = het_breuschpagan(residuals, sm.add_constant(X_test))
    dw_stat = durbin_watson(residuals)

    normality = "Normal" if shapiro_test.pvalue > 0.05 else "Not Normal"
    homoskedasticity = "Homoskedastic" if bp_test[1] > 0.05 else "Heteroskedastic"
    autocorr = "No Strong Autocorrelation" if 1.5 < dw_stat < 2.5 else "Possible Autocorrelation"

    print(f"\n{name} Diagnostic Tests:")
    print(f"Shapiro-Wilk p={shapiro_test.pvalue:.4f} → {normality}")
    print(f"Breusch-Pagan p={bp_test[1]:.4f} → {homoskedasticity}")
    print(f"Durbin-Watson={dw_stat:.2f} → {autocorr}")

    print(f"\n{name} Summary:")
    print(f"Train R²: {train_r2:.4f} | Test R²: {test_r2:.4f} | RMSE: {test_rmse:.4f}")
    
    return train_r2, test_r2, test_rmse


In [ ]:
#Ridge Regression
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
ridge_train_r2, ridge_test_r2, ridge_rmse = evaluate_model(
    ridge, "Ridge", X_train, X_test, y_train, y_test
)

In [ ]:
#Lasso Regression
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
lasso_train_r2, lasso_test_r2, lasso_rmse = evaluate_model(
    lasso, "Lasso", X_train, X_test, y_train, y_test
)

In [ ]:
#Model Comparison Summary
comparison = pd.DataFrame({
    "Model": ["Ridge", "Lasso"],
    "Train R²": [ridge_train_r2, lasso_train_r2],
    "Test R²": [ridge_test_r2, lasso_test_r2],
    "Test RMSE": [ridge_rmse, lasso_rmse]
})
print("\n=== Final Model Comparison ===")
print(comparison.round(4).to_string(index=False))